In [1]:
MODEL = "llama2:latest"

In [2]:
from langchain_ollama.llms import OllamaLLM
from langchain_community.embeddings import OllamaEmbeddings

model = OllamaLLM(model=MODEL)
embedding = OllamaEmbeddings()
# model.invoke('tell me a joke')


C:\Users\sagar\AppData\Local\Temp\ipykernel_11532\3187340122.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings()


In [3]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = model | parser
# chain.invoke('what is the capital of India')

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("sagar-manchakatla-resume.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-14T19:54:39+00:00', 'author': '', 'keywords': '', 'moddate': '2025-08-14T19:54:39+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'sagar-manchakatla-resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Sagar Manchakatla\n+91 9987799356 | sagarmanchakatla01@gmail.com | https://www.linkedin.com/in/sagar-manchakatla/\nEducation\nThadomal Shahani Engineering College Mumbai, India\nBachelor of Engineering in Computer Engineering; CGPA: 9.34 Aug. 2023 – May 2026\nGovernment Polytechnic Mumbai, India\nDiploma in Instrumentation Engineering; Percentage: 94% Aug. 2020 – May 2023\nExperience\nFull Stack Intern Aug. 2025 – Jul. 2025\nConverse AI Labs Remote\n• Developed and deployed backend services using FastAPI with scalable, low-latency, 

In [5]:
from langchain.prompts import PromptTemplate
template = """
    Answer the question based on the context below. If you can't 
    answer the question, reply "I don't know".

    Context: {context}

    Question: {question}
"""
prompt = PromptTemplate.from_template(template=template)

In [6]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'Hi there! My name is Santiago, just like my parents named me. Nice to meet you!'

In [ ]:
chain.input_schema.schema()

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding)

c:\OLLAMA\local-RAG\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
retriever = vectorstore.as_retriever()
retriever.invoke('Government Polytechnic')

[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-14T19:54:39+00:00', 'author': '', 'keywords': '', 'moddate': '2025-08-14T19:54:39+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'sagar-manchakatla-resume.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Sagar Manchakatla\n+91 9987799356 | sagarmanchakatla01@gmail.com | https://www.linkedin.com/in/sagar-manchakatla/\nEducation\nThadomal Shahani Engineering College Mumbai, India\nBachelor of Engineering in Computer Engineering; CGPA: 9.34 Aug. 2023 – May 2026\nGovernment Polytechnic Mumbai, India\nDiploma in Instrumentation Engineering; Percentage: 94% Aug. 2020 – May 2023\nExperience\nFull Stack Intern Aug. 2025 – Jul. 2025\nConverse AI Labs Remote\n• Developed and deployed backend services using FastAPI with scalable, low-latency, 

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    'How many internships i have completed',
    'How many projects i have completed'
    'What is my education'
]

for question in questions:
    print(f"Question : {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question : How many  internships i have completed
Answer: Based on the provided PDF document, you have completed 3 internships:

1. Full Stack Intern at Converse AI Labs from August to July of the following year (2025-2026).
2. MERN and Python Mentor at Static Int. Educare from July to December of the previous year (2024).
3. Projects intern at Endless Runner, MediaPipe, scikit-learn, OpenCV, and Unity from August to December of the previous year (2025).

Question : How many projects i have completedWhat is my education
Answer: Based on the provided PDF document, you have completed 5 projects:

1. Endless Runner
2. Connect.io
3. Travel-Ease

Your education is:

Thadomal Shahani Engineering College Mumbai, India
Bachelor of Engineering in Computer Engineering; CGPA: 9.34 Aug. 2023 – May 2026
Government Polytechnic Mumbai, India
Diploma in Instrumentation Engineering; Percentage: 94% Aug. 2020 – May 2023



In [12]:
chain.batch([{"question": q} for q in questions])


['Based on the provided PDF document, you have completed 3 internships:\n\n1. Full Stack Intern at Converse AI Labs from August 2025 to July 2025\n2. MERN and Python Mentor at Static Int. Educare from July 2024\n3. Project-based learning mentor at Endless Runner from January 2023 to April 2023\n\nTherefore, the answer to your question is "3 internships".',
 'Based on the provided PDF document, you have completed 7 projects. Your education is listed as "Thadomal Shahani Engineering College Mumbai, India" where you studied Computer Engineering and earned a Bachelor of Engineering degree with a CGPA of 9.34 from Aug. 2023 to May 2026, followed by a Diploma in Instrumentation Engineering from Government Polytechnic Mumbai, India with a percentage of 94 from Aug. 2020 to May 2023.']

In [ ]:
for s in chain.stream({"question": "What is the purpose of the course?"}):
    print(s, end="", flush=True)